# Using Distribution plot in MNIST

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/labmlai/inspectus/blob/main/notebooks/mnist.ipynb)

Here, we will use the MNIST dataset to train a simple Convolutional Neural Network (CNN) model. We will use the Distribution plot to visualize the training loss of the model.


In [ ]:
!pip install inspectus
!pip install --upgrade altair
!pip install labml

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Model was taken from the PyTorch example: [github](https://github.com/pytorch/examples/tree/main/mnist)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

#### Data Logger

A [data logger](data_logger.ipynb) is initialized for the path `mnist` and the data is cleared in case there are previous records.

In [ ]:
import inspectus.data_logger

step = 0
d_log = inspectus.data_logger.DataLogger('mnist')
d_log.clear()

#### Training

At each training step, we find individual losses for every sample in the batch. This is done by setting the `reduction` parameter to `none` in the loss function. The loss is then saved to the data logger.

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    global step
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if loss.item() < 2.0:
            with torch.no_grad():
                d_log.save('train_loss', F.nll_loss(output, target, reduction='none'), step)
                step += 1

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
torch.manual_seed(1)

if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

dataset1 = datasets.MNIST('../data', train=True, download=True,
                          transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                          transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=1, shuffle=False)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=0.05)

In [ ]:
for epoch in range(2):
    train(model, device, train_loader, optimizer, epoch)

In [ ]:
torch.save(model.state_dict(), "mnist_cnn.pt")

Here logged data are visualized using the Distribution plot. We also include the mean series in the plot and render the distribution with 5 bands

![Training Loss](../assets/mnist_train_loss.png)

In [ ]:
tl = d_log.read('train_loss')
inspectus.distribution({'train_loss': tl}, width=500, height=500)

The distribution is visualized using five bands, where the central band represents the median (50th percentile). The other bands correspond to specific percentiles: 0, 6.68, 15.87, 30.85, 50.00, 69.15, 84.13, 93.32, and 100.00.

![Training Loss Zoomed in](../assets/mnist_train_loss_tail.png)

From the plot, we observe that the median loss is relatively low, below 0.1. However, there are also significantly higher values like 9, which are outliers. These outliers increase the mean loss due to their larger magnitude of contribution, even though they represent a small portion of the data.

Typically, we plot the mean loss at each training step. We also render the mean loss on the plot. Upon close inspection, we notice that the mean is higher than the median, aligning with the third band. This difference is due to the influence of outliers.

In conclusion, the model's performance is actually better than what the mean loss indicates, excluding the outliers. To improve the model, we should investigate these outlier samples to determine if there are any issues with the data.


# Obtaining high loss samples

To obtain the samples with the highest loss, we first load the model and the test dataset. We then calculate the loss for each sample in the dataset and select the top 10 samples with the highest loss. Finally, we visualize these samples to understand why the model failed to predict them correctly.

In [ ]:
model = Net().to(device)
model.load_state_dict(torch.load("mnist_cnn.pt"))
model.eval()

In [ ]:
from torchvision import datasets

torch.manual_seed(1)

device = torch.device("mps")

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

dataset = datasets.MNIST('../data', download=True, train=False,
                          transform=transform)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=100000)

In [ ]:
data, target = next(iter(train_loader))
data = data.to(device)
target = target.to(device)

output = model(data.to(device))
loss = F.nll_loss(output, target.to(device), reduction='none')

In [ ]:
import matplotlib.pyplot as plt

top_losses = torch.topk(loss, 10)
fig, ax = plt.subplots(1, 10, figsize=(20, 5))
for i in range(10):
    ax[i].imshow(data[top_losses.indices[i]].cpu().numpy().reshape(28, 28), cmap='gray')
    ax[i].set_title(f"Ground truth: {target[top_losses.indices[i]].item()}\nPrediction: {output[top_losses.indices[i]].argmax().item()}")

![High Loss Samples](../assets/mnist_outlier_samples.png)